In [29]:
!pip install qiskit[all]

In [73]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [72]:
def deutsch_jozsa_algorithm(n, oracle_op):

    x = QuantumRegister(n, "x")
    y = QuantumRegister(1, "y")
    c = ClassicalRegister(n, "c")
    qc = QuantumCircuit(x, y, c)

    qc.x(y)

    qc.h(x)
    qc.h(y)
    qc.barrier()

    qc.unitary(oracle_op, x[:] + [y[0]], label="Uf")
    qc.barrier()

    qc.h(x)
    qc.barrier()

    qc.measure(x, c)

    return qc


In [81]:
def create_oracle(n, func_type):

    qc = QuantumCircuit(n + 1)

    if func_type == 'constant_zero':
        # Do nothing: f(x) = 0
        pass
    elif func_type == 'constant_one':
        # Flip the output qubit unconditionally: f(x) = 1
        qc.x(n)
    elif func_type == 'balanced_xor':
        # f(x) = x0 XOR x1 XOR ... XOR xn-1
        for i in range(n):
            qc.cx(i, n)
    else:
        raise ValueError("Unknown function type")

    return Operator(qc)

In [86]:
def run_and_classify(qc, label, shots=1024):
    print(f"\n{label} - Circuit Diagram:")
    print(qc.draw())

    sim = AerSimulator()
    result = sim.run(qc, shots=shots).result()
    counts = result.get_counts()
    """Did every single measurement (out of shots) return '000'?"

        If yes → the function is constant

        If not → the function is balanced
    """
    if counts.get("0" * qc.num_clbits, 0) == shots:
        print(f"{label}: Function is CONSTANT")
    else:
        print(f"{label}: Function is BALANCED")

    return counts

In [87]:
n = 3  # number of input bits

# Constant function f(x) = 0
oracle_const_0 = create_oracle(n, 'constant_zero')
qc_const_0 = deutsch_jozsa_algorithm(n, oracle_const_0)
run_and_classify(qc_const_0, "(f(x) = 0)")

# Constant function f(x) = 1
oracle_const_1 = create_oracle(n, 'constant_one')
qc_const_1 = deutsch_jozsa_algorithm(n, oracle_const_1)
run_and_classify(qc_const_1, "(f(x) = 1)")

# Balanced function f(x) = x0 XOR x1 XOR x2
oracle_bal = create_oracle(n, 'balanced_xor')
qc_bal = deutsch_jozsa_algorithm(n, oracle_bal)
run_and_classify(qc_bal, " (f(x) = x0 XOR x1 XOR x2)")


(f(x) = 0) - Circuit Diagram:
     ┌───┐      ░ ┌─────┐ ░ ┌───┐ ░ ┌─┐      
x_0: ┤ H ├──────░─┤0    ├─░─┤ H ├─░─┤M├──────
     ├───┤      ░ │     │ ░ ├───┤ ░ └╥┘┌─┐   
x_1: ┤ H ├──────░─┤1    ├─░─┤ H ├─░──╫─┤M├───
     ├───┤      ░ │  Uf │ ░ ├───┤ ░  ║ └╥┘┌─┐
x_2: ┤ H ├──────░─┤2    ├─░─┤ H ├─░──╫──╫─┤M├
     ├───┤┌───┐ ░ │     │ ░ └───┘ ░  ║  ║ └╥┘
  y: ┤ X ├┤ H ├─░─┤3    ├─░───────░──╫──╫──╫─
     └───┘└───┘ ░ └─────┘ ░       ░  ║  ║  ║ 
c: 3/════════════════════════════════╩══╩══╩═
                                     0  1  2 
(f(x) = 0): Function is CONSTANT

(f(x) = 1) - Circuit Diagram:
     ┌───┐      ░ ┌─────┐ ░ ┌───┐ ░ ┌─┐      
x_0: ┤ H ├──────░─┤0    ├─░─┤ H ├─░─┤M├──────
     ├───┤      ░ │     │ ░ ├───┤ ░ └╥┘┌─┐   
x_1: ┤ H ├──────░─┤1    ├─░─┤ H ├─░──╫─┤M├───
     ├───┤      ░ │  Uf │ ░ ├───┤ ░  ║ └╥┘┌─┐
x_2: ┤ H ├──────░─┤2    ├─░─┤ H ├─░──╫──╫─┤M├
     ├───┤┌───┐ ░ │     │ ░ └───┘ ░  ║  ║ └╥┘
  y: ┤ X ├┤ H ├─░─┤3    ├─░───────░──╫──╫──╫─
     └───┘└───┘ ░ └─────┘ ░    

{'111': 1024}